# The CFG Maze Game

## What is the Maze task?

The Maze Task is an experimental task used by many psycholinguists. In this task, the participant follows a sentence through a "maze". They will see two words at a time, and one of the words will be the correct word that continues the sentence. The participant will only reach the end of the sentence by selecting the correct word at each step. 

For example, for the sentence 'The rain fell silently', the participant may see something like this:

![Example maze task presentation from https://www.u.arizona.edu/~kforster/MAZE/how_it_works.htm](assets/example-maze-task.gif)

This task forces the reader into an incremental mode of processing in which each word must be fully integrated with the preceding context before the next word can be considered. Psycholinguists usually measure the reaction time needed to make the correct selection, which is associated with the difficulty of integrating the word with the previous words (the context). 

## The Maze task as a game

Psycholinguists use the maze task to study human sentence processing, but as you can see, for participants in the maze task, there is a goal (to get to the correct sentence) and there is a challenge (to avoid the wrong choice), so it can be something like a game. 

## A CFG Maze game

We just learned how to use CFG to generate sentences in Python. We can use that to generate trials in the Maze task. Because the sentences we generate from CFG don't have the 'points of processing difficulty' that psycholinguists study, so we can simply make it a small game (instead of a proper experiment that answers a research question). 

### Modify generate() to return both the sentence and the alternative option at each position

Remember in a maze task, at each word, the participant sees two options, one is correct, the other is not. We can modify generate() to give us both the sentence (the correct options) and the alternatives (the incorrect options). 

First let's consider what should be the alternative (incorrect) option? It has to be incorrect, so let's just say in the context of CFG, it cannot be a grammatical continuation of the last word. This means that the correct option needs to block all words that belong to the same node as the correct option, or any nodes that occupy the same syntactic position as the correct option's node. This is what's in `block_list`. 

In [4]:
import random

# our cfg dictionary
cfg = {"NP":[["D","N"],"cats","dogs"],"VP":[["V","NP"],"walk","sleep"],"S":[["NP","VP"]],"D":["the","a"],"N":["cat","dog"],"V":["love","tolerate"]}

# block_list has nodes as keys, and as values, a list of nodes that are identical to the key or occupy the same syntactic position as the key
block_list = {'NP':['NP','N'], 'N':['N','NP'], 'VP':['VP', 'V'], 'V':['V', 'VP'], 'D':['D']}

Next, let's modify the generate() function. We make it return a list of [correct, wrong, correct, wrong] for the randomly generated sentence. 

In [ ]:
def generate(cfg, node='S'):
    expansion = random.choice(cfg[node])
    if type(expansion) == list:
        return generate(cfg, expansion[0]) + generate(cfg, expansion[1])
    elif type(expansion) == str:
        # block the node that makes sense after the current node
        blocked_nodes = block_list[node]
        # get a dictionary of nodes:words that are not in blocked nodes
        option_pool = {key:cfg[key] for key in [i for i in list(cfg.keys()) if i not in blocked_nodes]}
        # get a list of options to choose from from the values of option_pool that are strings, but only if the element is a string (only if it's a word, not a list of nodes)
        options = [x for lis in list(option_pool.values()) for x in lis if type(x) == str]
        option = random.choice(options)
        return [expansion, option]

### Make the game

Now we can make the game! 

Think about what functions the game needs. First it needs a function to print the current sentence (whatever have been selected by the player) and the two options, and asks the player for a selection. We need a function to do something when the player selects the correct option, and a function that does something when the player selects the wrong option. 

Let's call the first funtion getSentence(). This needs to generate a sentence using generate(), and we also ask it to parse the list returned by generate() so it looks nicer.

In [ ]:
def getSentence():
    # global sents the variables to the global environment (outside the function), because we'll need these in other functions as well
    # sentence records the generated sentence
    # selected records what correct choices the player has selected
    # current_location records which words we're at in the sentence, in this function we just initialise it. 
    global sentence, selected, current_location
    # get a flat sentence list first
    sentence_flat = generate(cfg)
    # then parse the flat sentence into a dictionary of location:options
    sentence = {i:[sentence_flat[i*2], sentence_flat[(i*2)+1]] for i in range(int(len(sentence_flat)/2))}
    # record the first correct word (which we just show the player) in selected
    selected = [sentence[0][0]]
    current_location = 1
    # then get the options for the current location in the sentence
    scramble_options(current_location)

# Next, we need a function to scramble the options (so that the correct option is not always the left one...)
def scramble_options(current_location):
    global sentence, options
    # get options
    options = sentence[current_location].copy()
    # scramble the order of correct/incorrect options
    random.shuffle(options)

Next let's call this function printSentence(). It needs to do a few things:
- print the words selected by the player that's correct.

In [ ]:
def printSentence():
    global sentence
    print('----------')
    # print the (incomplete) sentence
    print(' '.join(selected))
    # print the options
    print('  '+options[0]+'        '+options[1])
    # ask for the input

In [ ]:
def getChoice():
  while True:
    try:
      choice = int(input('Which one do you choose?'))
      if choice in [1,2]:
        return choice
      else:
        print('Enter 1 for the left option, 2 for the right option.')
    except ValueError:
      print('Enter 1 for the left option, 2 for the right option.')

In [ ]:
def main():
    print('Welcome to the CFG Maze game!')
    print('Please select the option that makes the sentence grammatical.')
    printSentence()
    choice = getChoice()

In [7]:
sentence_flat = generate(cfg)
sentence = {i:[sentence_flat[i*2], sentence_flat[(i*2)+1]] for i in range(int(len(sentence_flat)/2))}

print(sentence)

{0: ['cats', 'tolerate'], 1: ['love', 'a'], 2: ['cats', 'a']}
